In [50]:
import pandas as pd

df = pd.read_csv("academy_awards.csv", encoding="ISO-8859-1")
print(df.head(3))
print(df.columns)

          Year               Category          Nominee  \
0  2010 (83rd)  Actor -- Leading Role    Javier Bardem   
1  2010 (83rd)  Actor -- Leading Role     Jeff Bridges   
2  2010 (83rd)  Actor -- Leading Role  Jesse Eisenberg   

                          Additional Info Won? Unnamed: 5 Unnamed: 6  \
0                      Biutiful {'Uxbal'}   NO        NaN        NaN   
1           True Grit {'Rooster Cogburn'}   NO        NaN        NaN   
2  The Social Network {'Mark Zuckerberg'}   NO        NaN        NaN   

  Unnamed: 7 Unnamed: 8 Unnamed: 9 Unnamed: 10  
0        NaN        NaN        NaN         NaN  
1        NaN        NaN        NaN         NaN  
2        NaN        NaN        NaN         NaN  
Index(['Year', 'Category', 'Nominee', 'Additional Info', 'Won?', 'Unnamed: 5',
       'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10'],
      dtype='object')


In [51]:
print(df["Year"].value_counts())

1941 (14th)      192
1942 (15th)      190
1943 (16th)      188
1944 (17th)      181
1945 (18th)      163
1940 (13th)      162
1939 (12th)      153
1998 (71st)      150
1938 (11th)      145
1954 (27th)      141
1995 (68th)      141
2009 (82nd)      140
1952 (25th)      138
1994 (67th)      138
1997 (70th)      137
2001 (74th)      137
2010 (83rd)      135
1953 (26th)      135
1937 (10th)      135
1956 (29th)      134
1955 (28th)      132
1964 (37th)      132
1996 (69th)      131
2006 (79th)      130
2005 (78th)      129
1990 (63rd)      129
1961 (34th)      129
2000 (73rd)      129
2004 (77th)      129
1966 (39th)      129
                ... 
1987 (60th)      119
2008 (81st)      118
1975 (48th)      118
1985 (58th)      116
1983 (56th)      116
1950 (23rd)      116
1936 (9th)       115
1958 (31st)      115
1948 (21st)      115
1974 (47th)      115
1946 (19th)      115
1972 (45th)      114
1947 (20th)      114
1989 (62nd)      114
1967 (40th)      113
1976 (49th)      113
1980 (53rd)  

In [52]:
import numpy as np

df["Year"] = df["Year"].str[0:4]
df["Year"] = df["Year"].astype(np.int64)

In [53]:
later_than_2000 = df[df["Year"] > 2000]

In [54]:
award_categories = ["Actor -- Leading Role", "Actor -- Supporting Role", "Actress -- Leading Role", "Actress -- Supporting Role"]
nominations = later_than_2000[later_than_2000["Category"].isin(award_categories)]

In [55]:
replace_dict = {"NO":0, "YES": 1}
pd.options.mode.chained_assignment = None
nominations["Won?"] = nominations["Won?"].map(replace_dict)
nominations["Won"] = nominations["Won?"]

In [56]:
cols_to_drop = ["Won?", "Unnamed: 5", "Unnamed: 6", "Unnamed: 7", "Unnamed: 8", "Unnamed: 9", "Unnamed: 10"]
final_nominations = nominations.drop(cols_to_drop, axis=1)

In [57]:
print(final_nominations.head(2))

   Year               Category        Nominee                Additional Info  \
0  2010  Actor -- Leading Role  Javier Bardem             Biutiful {'Uxbal'}   
1  2010  Actor -- Leading Role   Jeff Bridges  True Grit {'Rooster Cogburn'}   

   Won  
0    0  
1    0  


In [58]:
a_i = final_nominations["Additional Info"]
additional_info_one = a_i.str.rstrip("'}")
additional_info_two = additional_info_one.str.split(" {'")
final_nominations["Movie"] = additional_info_two.str[0]
final_nominations["Character"] = additional_info_two.str[1]
final_nominations = final_nominations.drop(["Additional Info"], axis=1)
print(final_nominations.head(5))

   Year               Category          Nominee  Won               Movie  \
0  2010  Actor -- Leading Role    Javier Bardem    0            Biutiful   
1  2010  Actor -- Leading Role     Jeff Bridges    0           True Grit   
2  2010  Actor -- Leading Role  Jesse Eisenberg    0  The Social Network   
3  2010  Actor -- Leading Role      Colin Firth    1   The King's Speech   
4  2010  Actor -- Leading Role     James Franco    0           127 Hours   

         Character  
0            Uxbal  
1  Rooster Cogburn  
2  Mark Zuckerberg  
3   King George VI  
4     Aron Ralston  


In [59]:
import sqlite3

conn = sqlite3.connect("nominations.db")
final_nominations.to_sql("nominations", conn, index=False)


In [65]:

q = "select * from nominations limit 10;"
result = conn.execute(q).fetchall()
print(result)

[(2010, 'Actor -- Leading Role', 'Javier Bardem', 0, 'Biutiful', 'Uxbal'), (2010, 'Actor -- Leading Role', 'Jeff Bridges', 0, 'True Grit', 'Rooster Cogburn'), (2010, 'Actor -- Leading Role', 'Jesse Eisenberg', 0, 'The Social Network', 'Mark Zuckerberg'), (2010, 'Actor -- Leading Role', 'Colin Firth', 1, "The King's Speech", 'King George VI'), (2010, 'Actor -- Leading Role', 'James Franco', 0, '127 Hours', 'Aron Ralston'), (2010, 'Actor -- Supporting Role', 'Christian Bale', 1, 'The Fighter', 'Dicky Eklund'), (2010, 'Actor -- Supporting Role', 'John Hawkes', 0, "Winter's Bone", 'Teardrop'), (2010, 'Actor -- Supporting Role', 'Jeremy Renner', 0, 'The Town', 'James Coughlin'), (2010, 'Actor -- Supporting Role', 'Mark Ruffalo', 0, 'The Kids Are All Right', 'Paul'), (2010, 'Actor -- Supporting Role', 'Geoffrey Rush', 0, "The King's Speech", 'Lionel Logue')]
